# Creating a Vertex AI Feature Store Using Pandas

### Learning objectives

1. Create Feature Store Resources.
2. Ingest Feature Values into Entity Type from a Pandas DataFrame.
3. Read/Online Serve Entity's Feature Values from Vertex AI Online Feature Store.
4. Batch Serve Feature Values from Vertex AI Feature Store.
5. Read the Updated Feature Values.


## Introduction

This notebook introduces Pandas support for Feature Store using Vertex AI SDK. For pre-requisites and introduction on Vertex AI SDK and Feature Store native support, please go through this [Colab notebook](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/sdk-feature-store.ipynb).

### Dataset

This notebook uses a movie recommendation dataset as an example throughout all the notebooks including this one. The original task is to train a model to predict if a user is going to watch a movie and serve the model online.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/sdk-feature-store-pandas.ipynb) -- try to complete that notebook first before reviewing this solution notebook.

## Before you begin

### Install additional packages

To run this notebook, you need to install the following packages for Python.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
    
! pip install -U {USER_FLAG} --upgrade google-cloud-aiplatform \
                                        google-cloud-bigquery \
                                        google-cloud-bigquery-storage \
                                        avro \
                                        pyarrow \
                                        pandas -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-recommendations-ai 0.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.8.1 which is incompatible.
apache-beam 2.40.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.
apache-beam 2.40.0 requires pyarrow<8.0.0,>=0.15.1, but you have pyarrow 9.0.0 which is incompatible.


### Restart the kernel

After you install the packages, you need to restart the notebook kernel so that it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # Replace with your project ID

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [3]:
REGION = "[your-region]"  # Replace with your region

if REGION == "[your-region]":
    REGION = "us-central1"

### Import libraries and define constants

In [4]:
import datetime

import pandas as pd
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

## Create Feature Store Resources

### Create Feature Store

The method to create a Feature Store returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Running the code cell creates a featurestore and prints the process logs.

In [5]:
movie_predictions_feature_store = aiplatform.Featurestore.create(
    featurestore_id="movie_predictions",
    online_store_fixed_node_count=1,
)

Creating Featurestore
Create Featurestore backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/5097191114432053248
Featurestore created. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions
To use this Featurestore in another session:
featurestore = aiplatform.Featurestore('projects/816732343711/locations/us-central1/featurestores/movie_predictions')


### Create Entity Types

Entity types can be created within the Featurestore class. Below, you create the `Users` entity type and `Movies` entity type. Process logs are printed in the output for each cell.

In [6]:
# TODO 1
# Create the Users entity type
users_entity_type = movie_predictions_feature_store.create_entity_type(
    entity_type_id="users",
    description="Users entity",
)

Creating EntityType
Create EntityType backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/operations/1886124580116889600
EntityType created. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users')


In [7]:
# TODO 2
# Create the Movies entity type
movies_entity_type = movie_predictions_feature_store.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
)

Creating EntityType
Create EntityType backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/operations/7891111733262024704
EntityType created. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies')


### Create Features
Features can be created within each entity type. Add defining features to the `Users` entity type and `Movies` entity type by using the following methods.

In [8]:
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
)

users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
)

users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
)

Creating Feature
Create Feature backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/5925853445868224512
Feature created. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/features/age
To use this Feature in another session:
feature = aiplatform.Feature('projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/features/age')
Creating Feature
Create Feature backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/897021511955644416
Feature created. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/features/gender
To use this Feature in another session:
feature = aiplatform.Feature('projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/features/gender')
Creating Feature
Create Feature backing LRO: projects/8

In [9]:
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description": "The title of the movie",
    },
    "genres": {
        "value_type": "STRING",
        "description": "The genre of the movie",
    },
    "average_rating": {
        "value_type": "DOUBLE",
        "description": "The average rating for the movie, range is [1.0-5.0]",
    },
}

In [10]:
movie_features = movies_entity_type.batch_create_features(
    feature_configs=movies_feature_configs,
)

Batch creating features EntityType entityType: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies
Batch create Features EntityType entityType backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/8192852908295847936
EntityType entityType Batch created features. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies


## Ingest Feature Values into Entity Type from a Pandas DataFrame

You need to ingest feature values into your entity type containing the features, so you can later `read` (online) or `batch serve` (offline) the feature values from the entity type. In this step, you will learn how to ingest feature values from a Pandas DataFrame into an entity type. We can also import feature values from BigQuery or Google Cloud Storage.


#### Get data from source files

In [11]:
GCS_USERS_AVRO_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
)
GCS_MOVIES_AVRO_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
)

USERS_AVRO_FN = "users.avro"
MOVIES_AVRO_FN = "movies.avro"

In [12]:
! gsutil cp $GCS_USERS_AVRO_URI $USERS_AVRO_FN
! gsutil cp $GCS_MOVIES_AVRO_URI $MOVIES_AVRO_FN

Copying gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro...
/ [1 files][  637.0 B/  637.0 B]                                                
Operation completed over 1 objects/637.0 B.                                      
Copying gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro...
/ [1 files][  599.0 B/  599.0 B]                                                
Operation completed over 1 objects/599.0 B.                                      


#### Load Avro Files into Pandas DataFrames

In [13]:
from avro.datafile import DataFileReader
from avro.io import DatumReader


class AvroReader:
    def __init__(self, data_file):
        self.avro_reader = DataFileReader(open(data_file, "rb"), DatumReader())

    def to_dataframe(self):
        records = [record for record in self.avro_reader]
        return pd.DataFrame.from_records(data=records)

In [14]:
users_avro_reader = AvroReader(data_file=USERS_AVRO_FN)
users_source_df = users_avro_reader.to_dataframe()
print(users_source_df)

   user_id   age  gender               liked_genres  \
0     dave   NaN    None  [Children's, Documentary]   
1    alice   NaN    None            [Drama, Comedy]   
2  charlie   NaN    None        [Sci-Fi, Animation]   
3      bob   NaN    None            [Action, Crime]   
4      eve  26.0    None                   [Horror]   
5      bob  35.0    Male        [Action, Adventure]   
6    alice  55.0  Female                    [Drama]   

                       update_time  
0 2021-08-20 20:58:22.261581+00:00  
1 2021-08-20 20:58:22.261581+00:00  
2 2021-08-20 20:58:22.261581+00:00  
3 2021-08-20 20:58:22.261581+00:00  
4 2021-08-20 20:58:22.261581+00:00  
5 2021-08-20 20:58:22.261581+00:00  
6 2021-08-20 20:58:22.261581+00:00  


In [15]:
movies_avro_reader = AvroReader(data_file=MOVIES_AVRO_FN)
movies_source_df = movies_avro_reader.to_dataframe()
print(movies_source_df)

   movie_id  average_rating                     title   genres  \
0  movie_02             4.2               The Shining   Horror   
1  movie_04             4.6           The Dark Knight   Action   
2  movie_03             4.5           Cinema Paradiso  Romance   
3  movie_01             4.9  The Shawshank Redemption    Drama   

                       update_time  
0 2021-08-20 20:44:11.094375+00:00  
1 2021-08-20 20:44:11.094375+00:00  
2 2021-08-20 20:44:11.094375+00:00  
3 2021-08-20 20:44:11.094375+00:00  


#### Ingest Feature Values into _Users_ Entity Type

In [16]:
# TODO 3
# Ingest Feature Values into Users Entity Type
users_entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=users_source_df,
    entity_id_field="user_id",
)

Importing EntityType feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users
Import EntityType feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/operations/1653626249353887744
EntityType feature values imported. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users


resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users

#### Ingest Feature Values into _Movies_ Entity Type

In [17]:
# TODO 4
# Ingest Feature Values into Movies Entity Type
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=movies_source_df,
    entity_id_field="movie_id",
)

Importing EntityType feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies
Import EntityType feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/operations/1220717735172898816
EntityType feature values imported. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies


resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies

## Read/Online Serve Entity's Feature Values from Vertex AI Online Feature Store

Feature Store allows [online serving](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)
which lets you read feature values for small batches of entities. It works well when you want to read values of selected features from an entity or multiple entities in an entity type.

In [18]:
users_read_df = users_entity_type.read(
    entity_ids=["dave", "alice", "charlie", "bob", "eve"],
)
print(users_read_df)

  entity_id   age               liked_genres  gender
0     alice  55.0                    [Drama]  Female
1       bob  35.0        [Action, Adventure]    Male
2   charlie   NaN        [Sci-Fi, Animation]    None
3      dave   NaN  [Children's, Documentary]    None
4       eve  26.0                   [Horror]    None


In [19]:
movies_read_df = movies_entity_type.read(
    entity_ids=["movie_01", "movie_02", "movie_03", "movie_04"],
    feature_ids=["title", "genres", "average_rating"],
)
print(movies_read_df)

  entity_id                     title   genres  average_rating
0  movie_01  The Shawshank Redemption    Drama             4.9
1  movie_02               The Shining   Horror             4.2
2  movie_03           Cinema Paradiso  Romance             4.5
3  movie_04           The Dark Knight   Action             4.6


## Batch Serve Feature Values from Vertex AI Feature Store

Batch Serving is used to fetch a large batch of feature values for high-throughput, and is typically used for training a model or batch prediction. In this section, you learn how to prepare training examples by using the Feature Store's batch serve function.

#### Read instances from source file

In [20]:
GCS_READ_INSTANCES_CSV_URI = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"
READ_INSTANCES_CSV_FN = "data.csv"

In [21]:
! gsutil cp $GCS_READ_INSTANCES_CSV_URI $READ_INSTANCES_CSV_FN

Copying gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv...
/ [1 files][  255.0 B/  255.0 B]                                                
Operation completed over 1 objects/255.0 B.                                      


#### Load CSV file into a Pandas DataFrame

In [22]:
# TODO 5
read_instances_df = pd.read_csv(READ_INSTANCES_CSV_FN)
print(read_instances_df)

   users    movies             timestamp
0  alice  movie_01  2021-09-15T08:28:14Z
1    bob  movie_02  2021-09-15T08:28:14Z
2    dav  movie_03  2021-09-15T08:28:14Z
3    eve  movie_04  2021-09-15T08:28:14Z
4  alice  movie_03  2021-09-14T09:35:15Z
5    bob  movie_04  2020-02-14T09:35:15Z


#### Change the Dtype of `Timestamp` to `Datetime64`

In [23]:
print("before: ", read_instances_df["timestamp"].dtype)
read_instances_df = read_instances_df.astype({"timestamp": "datetime64"})
print("after:  ", read_instances_df["timestamp"].dtype)

before:  object
after:   datetime64[ns]


#### Batch Serve Feature Values from Movie Predictions Feature Store

In [24]:
movie_predictions_df = movie_predictions_feature_store.batch_serve_to_df(
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["title", "average_rating", "genres"],
    },
    read_instances_df=read_instances_df,
)
movie_predictions_df

Serving Featurestore feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions
Serve Featurestore feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/9130868268184109056
Featurestore feature values served. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions


,timestamp,entity_type_users,age,gender,liked_genres,entity_type_movies,title,average_rating,genres
0,2020-02-14 09:35:15+00:00,bob,NaN,None,[],movie_04,None,NaN,None
1,2021-09-15 08:28:14+00:00,dav,NaN,None,[],movie_03,Cinema Paradiso,4.5,Romance
2,2021-09-15 08:28:14+00:00,eve,26.0,None,[Horror],movie_04,The Dark Knight,4.6,Action
3,2021-09-15 08:28:14+00:00,bob,35.0,Male,"[Action, Crime]",movie_02,The Shining,4.2,Horror
4,2021-09-15 08:28:14+00:00,alice,55.0,Female,[Drama],movie_01,The Shawshank Redemption,4.9,Drama
5,2021-09-14 09:35:15+00:00,alice,55.0,Female,[Drama],movie_03,Cinema Paradiso,4.5,Romance


## Read the Updated Feature Values

#### Feature Values from last ingestion
Recall read from the Entity Type shows Feature Values from the last ingestion.

In [25]:
print(movies_read_df)

  entity_id                     title   genres  average_rating
0  movie_01  The Shawshank Redemption    Drama             4.9
1  movie_02               The Shining   Horror             4.2
2  movie_03           Cinema Paradiso  Romance             4.5
3  movie_04           The Dark Knight   Action             4.6


#### Ingest updated Feature Values

In [26]:
update_movies_df = pd.DataFrame(
    data=[["movie_03", 4.3], ["movie_04", 4.8]],
    columns=["movie_id", "average_rating"],
)
print(update_movies_df)

   movie_id  average_rating
0  movie_03             4.3
1  movie_04             4.8


In [27]:
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating"],
    feature_time=datetime.datetime.now(),
    df_source=update_movies_df,
    entity_id_field="movie_id",
)

Importing EntityType feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies
Import EntityType feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/operations/7615406993573937152
EntityType feature values imported. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies


resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies

#### Latest Feature Values
Read from the Entity Type shows updated Feature values from the latest ingestion.

In [28]:
update_movies_read_df = movies_entity_type.read(
    entity_ids=["movie_01", "movie_02", "movie_03", "movie_04"],
    feature_ids=["title", "genres", "average_rating"],
)
print(update_movies_read_df)

  entity_id                     title   genres  average_rating
0  movie_01  The Shawshank Redemption    Drama             4.9
1  movie_02               The Shining   Horror             4.2
2  movie_03           Cinema Paradiso  Romance             4.3
3  movie_04           The Dark Knight   Action             4.8


## Point-in-Time Correctness

#### Missing data
Recall Batch Serve from the last ingestion has some missing data in it.

In [29]:
print(movie_predictions_df)

                  timestamp entity_type_users   age  gender     liked_genres  \
0 2020-02-14 09:35:15+00:00               bob   NaN    None               []   
1 2021-09-15 08:28:14+00:00               dav   NaN    None               []   
2 2021-09-15 08:28:14+00:00               eve  26.0    None         [Horror]   
3 2021-09-15 08:28:14+00:00               bob  35.0    Male  [Action, Crime]   
4 2021-09-15 08:28:14+00:00             alice  55.0  Female          [Drama]   
5 2021-09-14 09:35:15+00:00             alice  55.0  Female          [Drama]   

  entity_type_movies                     title  average_rating   genres  
0           movie_04                      None             NaN     None  
1           movie_03           Cinema Paradiso             4.5  Romance  
2           movie_04           The Dark Knight             4.6   Action  
3           movie_02               The Shining             4.2   Horror  
4           movie_01  The Shawshank Redemption             4.9    Dra

#### Backfill/Correct point-in-time data

In [30]:
backfill_users_df = pd.DataFrame(
    data=[["bob", 34, "Male", ["Drama"], "2020-02-13 09:35:15"]],
    columns=["user_id", "age", "gender", "liked_genres", "update_time"],
)
backfill_users_df = backfill_users_df.astype({"update_time": "datetime64"})
print(backfill_users_df)

  user_id  age gender liked_genres         update_time
0     bob   34   Male      [Drama] 2020-02-13 09:35:15


In [31]:
backfill_movies_df = pd.DataFrame(
    data=[["movie_04", 4.2, "The Dark Knight", "Action", "2020-02-13 09:35:15"]],
    columns=["movie_id", "average_rating", "title", "genres", "update_time"],
)
backfill_movies_df = backfill_movies_df.astype({"update_time": "datetime64"})
print(backfill_movies_df)

   movie_id  average_rating            title  genres         update_time
0  movie_04             4.2  The Dark Knight  Action 2020-02-13 09:35:15


#### Ingest backfilled/corrected point-in-time data from dataframe

In [32]:
users_entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=backfill_users_df,
    entity_id_field="user_id",
)

Importing EntityType feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users
Import EntityType feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users/operations/541237141393375232
EntityType feature values imported. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users


resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/users

In [33]:
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=backfill_movies_df,
    entity_id_field="movie_id",
)

Importing EntityType feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies
Import EntityType feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/operations/1467289814771433472
EntityType feature values imported. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies


resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions/entityTypes/movies

#### Latest ingestion with imputed missing data
Batch Serve from the latest ingestion with backfill/correction has reduced missing data.

In [34]:
backfill_movie_predictions_df = movie_predictions_feature_store.batch_serve_to_df(
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["title", "average_rating", "genres"],
    },
    read_instances_df=read_instances_df,
)
print(backfill_movie_predictions_df)

Serving Featurestore feature values: projects/816732343711/locations/us-central1/featurestores/movie_predictions
Serve Featurestore feature values backing LRO: projects/816732343711/locations/us-central1/featurestores/movie_predictions/operations/3882908064902283264
Featurestore feature values served. Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions
                  timestamp entity_type_users   age  gender     liked_genres  \
0 2021-09-15 08:28:14+00:00               dav   NaN    None               []   
1 2021-09-15 08:28:14+00:00               eve  26.0    None         [Horror]   
2 2020-02-14 09:35:15+00:00               bob  34.0    Male          [Drama]   
3 2021-09-15 08:28:14+00:00               bob  35.0    Male  [Action, Crime]   
4 2021-09-14 09:35:15+00:00             alice  55.0  Female          [Drama]   
5 2021-09-15 08:28:14+00:00             alice  55.0  Female          [Drama]   

  entity_type_movies                     titl

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

You can also keep the project but delete the featurestore by running the code below:

In [35]:
movie_predictions_feature_store.delete(force=True)

Deleting Featurestore : projects/816732343711/locations/us-central1/featurestores/movie_predictions
Delete Featurestore  backing LRO: projects/816732343711/locations/us-central1/operations/1667699998189420544
Featurestore deleted. . Resource name: projects/816732343711/locations/us-central1/featurestores/movie_predictions
